In [5]:
import pandas as pd
import glob

file_paths = [
    "metro-trips-2022-q1.csv", 
    "metro-trips-2022-q2.csv", 
    "metro-trips-2022-q3.csv", 
    "metro-trips-2022-q4.csv", 
    "metro-trips-2023-q1.csv", 
    "metro-trips-2023-q2.csv", 
    "metro-trips-2023-q3.csv", 
    "metro-trips-2023-q4.csv", 
    "metro-trips-2024-q1.csv", 
    "metro-trips-2024-q2.csv", 
    "metro-trips-2024-q3.csv"
]

# Read all data files and combine them into a single dataset
all_data = pd.concat([pd.read_csv(file) for file in file_paths], ignore_index=True)
all_data = all_data[all_data['duration'] >= 3]

# Define the latitude and longitude range for filtering conditions
lat_min, lat_max = 34.02, 34.12
lon_min, lon_max = -118.35, -118.22

# Filter the data based on the specified conditions
filtered_data = all_data[
    (all_data['start_lat'] >= lat_min) &
    (all_data['start_lat'] <= lat_max) &
    (all_data['start_lon'] >= lon_min) &
    (all_data['start_lon'] <= lon_max) ]

# Convert start_time to datetime and extract the hour
filtered_data['start_time'] = pd.to_datetime(filtered_data['start_time'])
filtered_data['hour'] = filtered_data['start_time'].dt.floor('h')

# Summing ride counts by start_station to identify stations with at least 5000 rides
station_counts = filtered_data['start_station'].value_counts()
stations_with_enough_rides = station_counts[station_counts >= 4000].index

# Further filter the data to include only stations with at least 5000 rides
filtered_data = filtered_data[filtered_data['start_station'].isin(stations_with_enough_rides)]

# Group by hour and start_station to calculate the ride count
hourly_rides = filtered_data.groupby(['hour', 'start_station']).size().reset_index(name='ride_count')

# Save the result to a CSV file
#hourly_rides.to_csv("hourly_rides_filtered.csv", index=False)

hourly_rides.head(10)


/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_67024/74761937.py:19: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.concat([pd.read_csv(file) for file in file_paths], ignore_index=True)
/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_67024/74761937.py:19: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.concat([pd.read_csv(file) for file in file_paths], ignore_index=True)
/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_67024/74761937.py:19: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.concat([pd.read_csv(file) for file in file_paths], ignore_index=True)
/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_67024/74761937.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

,hour,start_station,ride_count
0,2022-01-01 00:00:00,3005,1
1,2022-01-01 01:00:00,4512,1
2,2022-01-01 02:00:00,3006,1
3,2022-01-01 02:00:00,3008,2
4,2022-01-01 02:00:00,3031,1
5,2022-01-01 03:00:00,4512,1
6,2022-01-01 04:00:00,3005,1
7,2022-01-01 04:00:00,3035,1
8,2022-01-01 05:00:00,3030,1
9,2022-01-01 05:00:00,3035,1


In [41]:
hourly_rides_pivot = hourly_rides.pivot_table(
    index='hour', 
    columns='start_station', 
    values='ride_count', 
    aggfunc='sum', 
    fill_value=0
)
hourly_rides_pivot.head()

start_station,3005,3006,3007,3008,3014,3018,3019,3022,3026,3027,...,4479,4491,4509,4512,4516,4524,4578,4584,4587,4627
hour,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-01 01:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2022-01-01 02:00:00,0,1,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-01 03:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2022-01-01 04:00:00,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
num_columns = hourly_rides_pivot.shape[1]
print(f"总共有 {num_columns} 列。")


总共有 50 列。


In [26]:
# 将索引重置为普通列
hourly_rides_flat = hourly_rides_pivot.reset_index()


In [32]:
hourly_rides_flat.to_csv('hourly_rides_flat.csv', index=False)


In [45]:
# 确保两个数据框的时间列格式一致
hourly_rides_flat['hour'] = pd.to_datetime(hourly_rides_flat['hour'])
weather_data['time'] = pd.to_datetime(weather_data['time'])

# 按时间列合并
merged_data = pd.merge(hourly_rides_flat, weather_data, left_on='hour', right_on='time', how='inner')

# 删除多余的时间列（可选）
merged_data.drop(columns=['time'], inplace=True)

# 删除前两列
merged_data = merged_data.iloc[:, 2:]

merged_data.head()

,hour,3005,3006,3007,3008,3014,3018,3019,3022,3026,...,4516,4524,4578,4584,4587,4627,temperature_2m (°C),apparent_temperature (°C),rain (mm),wind_speed_100m (km/h)
0,2022-01-01 00:00:00,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,7.7,4.7,0.0,17.6
1,2022-01-01 01:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,6.0,3.1,0.0,8.6
2,2022-01-01 02:00:00,0,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,4.8,2.0,0.0,11.7
3,2022-01-01 03:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4.7,1.7,0.0,10.1
4,2022-01-01 04:00:00,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3.3,0.4,0.0,6.2


In [46]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# 确保 'hour' 列为 datetime 格式
merged_data['hour'] = pd.to_datetime(merged_data['hour'])

# 提取周期性特征
merged_data['hour_of_day'] = merged_data['hour'].dt.hour
merged_data['weekday'] = merged_data['hour'].dt.weekday
merged_data['month'] = merged_data['hour'].dt.month

# 周期性编码
merged_data['hour_sin'] = np.sin(2 * np.pi * merged_data['hour_of_day'] / 24)
merged_data['hour_cos'] = np.cos(2 * np.pi * merged_data['hour_of_day'] / 24)
merged_data['weekday_sin'] = np.sin(2 * np.pi * merged_data['weekday'] / 7)
merged_data['weekday_cos'] = np.cos(2 * np.pi * merged_data['weekday'] / 7)
merged_data['month_sin'] = np.sin(2 * np.pi * merged_data['month'] / 12)
merged_data['month_cos'] = np.cos(2 * np.pi * merged_data['month'] / 12)

# 添加高峰/非高峰阶段特征
def classify_peak_hour(hour):
    if 12 <= hour <= 18:  # 高峰期
        return 'peak'
    elif 0 <= hour <= 6:  # 非高峰期
        return 'off_peak'
    else:  # 其他阶段
        return 'other'

merged_data['peak_stage'] = merged_data['hour_of_day'].apply(classify_peak_hour)
cat_features = ['peak_stage']

encoder = OneHotEncoder(sparse_output=False)
encoded_cat = encoder.fit_transform(merged_data[cat_features])
encoded_cat_columns = encoder.get_feature_names_out(cat_features)

# 拼接数值特征、周期性特征和 One-Hot 编码特征
encoded_cat_df = pd.DataFrame(encoded_cat, columns=encoded_cat_columns, index=merged_data.index)
merged_data = pd.concat([merged_data, encoded_cat_df], axis=1)

# 删除中间特征列（如 'hour_of_day', 'weekday', 'month'），保留其他原始列和新特征
merged_data.drop(columns=['hour_of_day', 'weekday', 'month','peak_stage'], inplace=True)

# 查看结果
merged_data.head()




,hour,3005,3006,3007,3008,3014,3018,3019,3022,3026,...,wind_speed_100m (km/h),hour_sin,hour_cos,weekday_sin,weekday_cos,month_sin,month_cos,peak_stage_off_peak,peak_stage_other,peak_stage_peak
0,2022-01-01 00:00:00,1,0,0,0,0,0,0,0,0,...,17.6,0.000000,1.000000,-0.974928,-0.222521,0.5,0.866025,1.0,0.0,0.0
1,2022-01-01 01:00:00,0,0,0,0,0,0,0,0,0,...,8.6,0.258819,0.965926,-0.974928,-0.222521,0.5,0.866025,1.0,0.0,0.0
2,2022-01-01 02:00:00,0,1,0,2,0,0,0,0,0,...,11.7,0.500000,0.866025,-0.974928,-0.222521,0.5,0.866025,1.0,0.0,0.0
3,2022-01-01 03:00:00,0,0,0,0,0,0,0,0,0,...,10.1,0.707107,0.707107,-0.974928,-0.222521,0.5,0.866025,1.0,0.0,0.0
4,2022-01-01 04:00:00,1,0,0,0,0,0,0,0,0,...,6.2,0.866025,0.500000,-0.974928,-0.222521,0.5,0.866025,1.0,0.0,0.0


In [47]:
merged_data.to_csv('pivot_50_peak.csv',index=False)